In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
os.chdir("..")
print(os.getcwd())
sys.path.insert(0, os.getcwd())

/home/gy46/NFGP


In [2]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [3]:
!ls -d logs/*/

logs/armidillo_back_s1e-1_b1e-3_2021-Dec-02-22-15-32/
logs/armidillo_ballet_s1e-1_b1e-3_2021-Dec-06-23-46-47/
logs/armidillo_front_s1e-1_b1e-3_2021-Dec-02-22-15-34/
logs/armidillo_handsup_s1e-1_b1e-3_2021-Dec-06-23-47-11/
logs/armidillo_LR_s1e-1_b1e-3_2021-Dec-02-22-15-31/
logs/armidillo_RL_s1e-1_b1e-3_2021-Dec-02-22-15-31/
logs/bar_twist_s1e-1_b1e-3_2021-Nov-26-17-52-13/
logs/bar_twist_s1e-1_b1e-5_2021-Nov-28-12-13-04/
logs/cactus_translate_s1e-1_b1e-3_2021-Nov-28-12-13-06/
logs/cactus_xrotate_s1e-1_b1e-3_2021-Nov-28-12-13-11/
logs/cactus_yrotate_s1e-1_b1e-3_2021-Nov-28-12-13-12/
logs/checkpoints/
logs/cylinder_rotate_s1e-1_b1e-3_2021-Nov-23-09-45-21/
logs/cylinder_translate_s1e-1_b1e-3_2021-Nov-23-09-45-39/
logs/dino_bend_s0_b1e-3_2021-Nov-21-16-47-21/
logs/dino_bend_s1e-1_b0_2021-Nov-21-16-47-17/
logs/dino_bend_s1e-1_b1e-3_2021-Nov-21-16-43-33/
logs/dino_bend_s1e-1_b1e-3_2021-Nov-23-09-45-52/
logs/dino_bend_s1e-1_b1e-3_hthr1e-4_2021-Nov-21-16-47-25/
logs/images/
logs/jolteon_jump_s1

In [5]:
log_path = "logs/armidillo_ballet_s1e-1_b1e-3_2021-Dec-06-23-46-47/"

In [6]:
import os.path as osp
from utils import load_imf

net, cfg = load_imf(
    log_path, 
    return_cfg=True)
net, cfg

logs/armidillo_ballet_s1e-1_b1e-3_2021-Dec-06-23-46-47/checkpoints/epoch_199_iters_20000.pt
Original Decoder:
Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
  (act): Sine()
)
Net(
  (blocks): ModuleList(
    (0): InvertibleResBlockLinear(
      (pos_enc): LipBoundedPosEnc()
      (blocks): ModuleList(
        (0): Linear(in_features=33, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
        (2): Linear(in_features=256, out_features=3, bias=True)
      )
      (act): ELU(alpha=1.0)
    )
    (1): InvertibleResBlockLinear(
      (po

100%|██████████| 1678/1678 [00:14<00:00, 116.35it/s]


203.43166 28.834208


(DeformationWrapper(
   (orig): Net(
     (blocks): ModuleList(
       (0): Linear(in_features=3, out_features=512, bias=True)
       (1): Linear(in_features=512, out_features=512, bias=True)
       (2): Linear(in_features=512, out_features=512, bias=True)
       (3): Linear(in_features=512, out_features=512, bias=True)
       (4): Linear(in_features=512, out_features=512, bias=True)
       (5): Linear(in_features=512, out_features=512, bias=True)
       (6): Linear(in_features=512, out_features=1, bias=True)
     )
     (act): Sine()
   )
   (deform): Net(
     (blocks): ModuleList(
       (0): InvertibleResBlockLinear(
         (pos_enc): LipBoundedPosEnc()
         (blocks): ModuleList(
           (0): Linear(in_features=33, out_features=256, bias=True)
           (1): Linear(in_features=256, out_features=256, bias=True)
           (2): Linear(in_features=256, out_features=3, bias=True)
         )
         (act): ELU(alpha=1.0)
       )
       (1): InvertibleResBlockLinear(
        

# Load dataset

In [8]:
import trimesh
import importlib
data_lib = importlib.import_module(cfg.data.type)
loader = data_lib.get_data_loaders(cfg.data, None)['train_loader']
for data in loader:
    break
    
print(data.keys())
handles = data['handles'].view(-1, 3).detach().cpu().numpy()
targets = data['targets'].view(-1, 3).detach().cpu().numpy()
print(handles.shape, targets.shape)

if 'gtr_verts' in data and 'gtr_faces' in data:
    p = mp.plot(data['gtr_verts'].cpu().view(-1, 3).numpy(), 
            data['gtr_faces'].cpu().view(-1, 3).numpy())
    p.add_points(handles, shading={'point_color': 'red', 'point_size': 0.05})
    p.add_points(targets, shading={'point_color': 'blue', 'point_size': 0.05})

dict_keys(['idx', 'handles', 'targets'])
(22, 3) (22, 3)


# Get the output Mesh

In [9]:
from trainers.utils.vis_utils import imf2mesh
res = 256
out_mesh = imf2mesh(net, res=res, normalize=True, bound=1.2)

100%|██████████| 1678/1678 [00:41<00:00, 40.55it/s]


247.11562 32.220642


In [10]:
p = mp.plot(out_mesh.vertices.astype(np.float32), out_mesh.faces.astype(np.float32)) 
p.add_points(handles, shading={'point_color': 'red', 'point_size': 0.05})
p.add_points(targets, shading={'point_color': 'blue', 'point_size': 0.05})

/home/gy46/anaconda3/envs/IGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/gy46/anaconda3/envs/IGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.010060…

2

In [ ]:
# Visualize the original neural fields